# 🎬 UVG MAX – Premium AI Video Generator

Welcome to **UVG MAX**, a professional-grade AI video generation engine!

### What this notebook does:
1. **Clones** the UVG MAX repository
2. **Installs** all dependencies
3. **Configures** your API keys
4. **Generates** a cinematic video from your prompt
5. **Displays** the final video and thumbnail

### Requirements:
- Pexels API Key (free at [pexels.com/api](https://www.pexels.com/api/))
- Pixabay API Key (free at [pixabay.com/api](https://pixabay.com/api/docs/))
- Gemini API Key (free at [makersuite.google.com](https://makersuite.google.com/app/apikey))
- Azure TTS Key (free tier at [azure.microsoft.com](https://azure.microsoft.com/en-us/products/ai-services/text-to-speech))

---
**Run each cell in order ⬇️**

## 📦 Step 1: Clone Repository

In [ ]:
# Clone the UVG MAX repository
!git clone https://github.com/ShyaBubu/uvg-max-engine.git
%cd uvg-max-engine
print("✅ Repository cloned successfully!")

## 📥 Step 2: Install Dependencies

In [ ]:
# Install required packages
!pip install -q -r requirements.txt

# Verify FFmpeg is available
!ffmpeg -version | head -1

print("✅ Dependencies installed!")

## 🔑 Step 3: Configure API Keys

Enter your API keys below. They will be stored as environment variables for this session only.

In [ ]:
import os
from getpass import getpass

print("🔐 Enter your API keys (they won't be displayed):")
print("-" * 50)

# Media Search APIs
os.environ["PEXELS_KEY"] = getpass("Pexels API Key: ")
os.environ["PIXABAY_KEY"] = getpass("Pixabay API Key: ")

# AI Services
os.environ["GEMINI_API_KEY"] = getpass("Gemini API Key: ")

# Azure TTS
os.environ["AZURE_TTS_KEY"] = getpass("Azure TTS Key: ")
os.environ["AZURE_TTS_REGION"] = input("Azure TTS Region (e.g., eastus): ") or "eastus"

print("\n✅ API keys configured!")

## 🎬 Step 4: Generate Your Video

Customize the prompt and settings below, then run the cell!

In [ ]:
from uvg_core.orchestrator import Orchestrator
from uvg_core.config import UVGConfig
from uvg_core.script_generator import ScriptGenerator

# ============================================
# 🎨 CUSTOMIZE YOUR VIDEO HERE
# ============================================
VIDEO_TITLE = "Ocean Sunset"  # @param {type:"string"}
VIDEO_PROMPT = "A cinematic ocean sunset with orange and pink hues reflecting on calm waves."  # @param {type:"string"}
TOTAL_DURATION = 15  # @param {type:"number"}
STYLE_PRESET = "cinematic"  # @param ["cinematic", "motivational", "tiktok", "travel", "romantic"]
# ============================================

print(f"🎬 Generating: {VIDEO_TITLE}")
print(f"📝 Prompt: {VIDEO_PROMPT}")
print(f"⏱️ Duration: {TOTAL_DURATION}s")
print(f"🎨 Style: {STYLE_PRESET}")
print("-" * 50)

# Load configuration
config = UVGConfig.from_env()
config.apply_preset(STYLE_PRESET)

# Generate script from prompt
print("\n📝 Generating script...")
script_gen = ScriptGenerator(gemini_api_key=os.environ.get("GEMINI_API_KEY", ""))
script = script_gen.generate_script(
    topic=VIDEO_PROMPT,
    num_scenes=max(3, TOTAL_DURATION // 5),
    tone="cinematic"
)

print(f"Generated {len(script.scenes)} scenes")
for i, scene in enumerate(script.scenes):
    print(f"  Scene {i+1}: {scene.text[:50]}...")

# Initialize orchestrator
print("\n🎥 Processing scenes...")
orch = Orchestrator(config=config.to_dict())

# Run pipeline
result = orch.run_pipeline(
    script={
        "title": VIDEO_TITLE,
        "scenes": [s.to_dict() for s in script.scenes]
    }
)

print("\n" + "=" * 50)
if result.get("success"):
    print("✅ VIDEO GENERATED SUCCESSFULLY!")
    print(f"📁 Output: {result.get('output_path')}")
else:
    print("⚠️ Generation completed with some issues")
    print(f"Completed scenes: {result.get('scenes', [])}")

## 🎥 Step 5: Watch Your Video

In [ ]:
from IPython.display import Video, display
import os

# Find the output video
video_path = "uvg_output/final/final.mp4"

if os.path.exists(video_path):
    print("🎬 Your generated video:")
    display(Video(video_path, embed=True, width=400))
else:
    # Try to find any mp4 in output
    import glob
    videos = glob.glob("uvg_output/**/*.mp4", recursive=True)
    if videos:
        print(f"🎬 Found video: {videos[0]}")
        display(Video(videos[0], embed=True, width=400))
    else:
        print("⚠️ No video found. Check the generation logs above.")

## 🖼️ Step 6: View Thumbnail

In [ ]:
from PIL import Image
from IPython.display import display
import glob

# Find thumbnail
thumb_paths = glob.glob("uvg_output/thumbnails/*.jpg") + glob.glob("uvg_output/**/*thumb*.jpg", recursive=True)

if thumb_paths:
    print("🖼️ Generated thumbnail:")
    img = Image.open(thumb_paths[0])
    display(img)
else:
    print("ℹ️ No thumbnail found. You can generate one manually.")

## 💾 Step 7: Download Your Files

In [ ]:
from google.colab import files
import glob

print("📥 Available files for download:")

# List output files
output_files = glob.glob("uvg_output/**/*", recursive=True)
output_files = [f for f in output_files if not os.path.isdir(f)]

for f in output_files[:10]:
    print(f"  - {f}")

# Download the main video
videos = [f for f in output_files if f.endswith('.mp4')]
if videos:
    print(f"\n⬇️ Downloading: {videos[0]}")
    files.download(videos[0])

---

## 🎉 Done!

You've successfully generated a cinematic AI video with UVG MAX!

### Next Steps:
- Try different prompts and style presets
- Adjust the duration for longer/shorter videos
- Star ⭐ the repo if you found it useful!

**GitHub**: [github.com/ShyaBubu/uvg-max-engine](https://github.com/ShyaBubu/uvg-max-engine)